IMPORT STATEMENTS

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from statistics import mean
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix

from matplotlib.colors import ListedColormap

from sklearn.metrics import classification_report

IMPORT DATASET

In [23]:
datasets = pd.read_csv('data-class.csv')
datasets.describe()

,IDType,NOII_type,NOAM_type,NOCS_type,NOM_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,DIT_type,...,number_abstract_methods,number_not_abstract_not_final_methods,number_static_methods,number_final_static_methods,number_final_not_static_methods,number_not_final_static_methods,number_not_final_not_static_methods,number_standard_design_methods,number_constructor_DefaultConstructor_methods,number_constructor_NotDefaultConstructor_methods
count,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,...,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000
mean,15339.166667,0.500000,6.083333,1.247619,21.861905,15.314286,8.376190,15.778571,14.142857,1.850000,...,0.069048,21.135714,2.240476,0.121429,0.690476,2.119048,19.085714,14.645238,0.154762,1.133333
std,14493.493359,0.973393,9.384912,4.438454,33.939156,34.857450,14.083671,29.836324,22.098361,1.410454,...,0.486077,33.220076,19.544009,0.779828,5.157603,19.428472,26.859793,29.472536,0.362109,1.226724
min,25.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2071.750000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,1.000000,3.000000,1.000000,...,0.000000,4.750000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000
50%,11974.500000,0.000000,3.000000,0.000000,9.000000,0.000000,1.000000,4.000000,6.000000,1.000000,...,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,8.500000,2.000000,0.000000,1.000000
75%,24549.000000,1.000000,8.000000,0.000000,25.250000,13.000000,11.000000,18.000000,15.250000,2.000000,...,0.000000,23.000000,0.000000,0.000000,0.000000,0.000000,22.000000,16.250000,0.000000,1.000000
max,58934.000000,11.000000,89.000000,46.000000,395.000000,332.000000,77.000000,394.000000,155.000000,7.000000,...,6.000000,394.000000,387.000000,9.000000,65.000000,386.000000,188.000000,392.000000,1.000000,12.000000


In [24]:
X = datasets.iloc[:, '3':'57'].values
Y = datasets.iloc[:, [58]].values
print(X.shape)

kf = KFold(n_splits = 5)
f1score = []
f1scoreBagging = []
for train_index , test_index in kf.split(X):
    X_Train,X_Test = X[train_index],X[test_index]
    Y_Train,Y_Test = Y[train_index],Y[test_index]
    
    #Feature Scaling
    sc_X = StandardScaler()
    X_Train = sc_X.fit_transform(X_Train)
    X_Test = sc_X.transform(X_Test)
    
    #FITTING THE CLASSIFIER INTO THE TRAINING SET
    
    classifier = tree.DecisionTreeClassifier()
    classifierBagging = BaggingClassifier(n_estimators = 125 , random_state = 0)
    classifier.fit(X_Train,Y_Train)
    classifierBagging.fit(X_Train,Y_Train)
    
    #PREDICTING THE TEST SET RESULTS
    Y_Pred = classifier.predict(X_Test)
    Y_Pred_Bagging = classifierBagging.predict(X_Test)
    
    #Making confusion matrix
    cm = confusion_matrix(Y_Test, Y_Pred)
    cm_bagging = confusion_matrix(Y_Test,Y_Pred_Bagging)
    print("W/O", cm,"\n")
    print("W:", cm_bagging)
    
    X_Set, Y_Set = X_Test, Y_Test
    
    #Plotting
    X1, X2 = np.meshgrid(np.arange(start = X_Set[:, 0].min() - 1, stop = X_Set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_Set[:, 1].min() - 1, stop = X_Set[:, 1].max() + 1, step = 0.01))

    #Without bagging
    plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
                 alpha = 0.75, cmap = ListedColormap(('pink', 'yellow')))
    plt.xlim(X1.min(), X1.max())
    plt.ylim(X2.min(), X2.max())
    for i, j in enumerate(np.unique(Y_Set)):
        plt.scatter(X_Set[Y_Set == j, 0], X_Set[Y_Set == j, 1],
                    c = ListedColormap(('red', 'green'))(i), label = j)
    plt.title('Decision Tree Classifier (Test set)')
    plt.xlabel('Age')
    plt.ylabel('Estimated Salary')
    plt.legend()
    plt.show()
    
    #With Bagging
    plt.contourf(X1, X2, classifierBagging.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
                 alpha = 0.75, cmap = ListedColormap(('pink', 'yellow')))
    plt.xlim(X1.min(), X1.max())
    plt.ylim(X2.min(), X2.max())
    for i, j in enumerate(np.unique(Y_Set)):
        plt.scatter(X_Set[Y_Set == j, 0], X_Set[Y_Set == j, 1],
                    c = ListedColormap(('red', 'green'))(i), label = j)
    plt.title('Bagged Decision Tree (Test set)')
    plt.xlabel('Age')
    plt.ylabel('Estimated Salary')
    plt.legend()
    plt.show()
    
    #Calculating the F1 Score
    varBag = classification_report(Y_Test,Y_Pred_Bagging,output_dict=True)['weighted avg']['f1-score']
    f1scoreBagging.append(varBag)
    
    var = classification_report(Y_Test,Y_Pred,output_dict=True)['weighted avg']['f1-score']
    f1score.append(var)
    
print("Mean F1-Score(Bagging) :", mean(f1scoreBagging),"\n")
print("Mean F1-Score(Without) :", mean(f1score))

TypeError: cannot do positional indexing on Index with these indexers [3] of type str